<a href="https://colab.research.google.com/github/SrBliss/CometML-NLP_IMDB/blob/main/Comet_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required packages

In [1]:
!pip install comet_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412 kB 5.1 MB/s 
     |████████████████████████████████| 162 kB 73.1 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 495 kB 54.6 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 140 kB 52.7 MB/s 
     |████████████████████████████████| 127 kB 49.9 MB/s 
     |████████████████████████████████| 162 kB 56.8 MB/s 
     |████████████████████████████████| 158 kB 57.5 MB/s 
     |████████████████████████████████| 157 kB 35.2 MB/s 
     |████████████████████████████████| 157 kB 48.6 MB/s 
     |████████████████████████████████| 157 kB 47.3 MB/s 
     |████████████████████████████████| 157 kB 46.1 MB/s 
     |████████████████████████████████| 157 kB 45.4 MB/s 
     |████████████████████████████████| 157 kB 57.4 MB/s 
     |██████████████████████████████

## Importing required packages

In [2]:
# import logging
import os

from comet_ml import Experiment
from comet_ml import config
from comet_ml import API

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Load data and create partitions (train, validation, test)

*This cell takes about 1 minute to execute*

In [3]:
# Load data
train_data, test_data = tfds.load(name="imdb_reviews",
                                  split=["train", "test"], 
                                  batch_size=-1, 
                                  as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)
    
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLBNXD2/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLBNXD2/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLBNXD2/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## My Experiment: Build, Train and Track the Movie Reviews Model 

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
# TO DO: API on config file or on environment variable
# os.environ['COMET_API_KEY'] = "EpnUTFvwqbgvmSpjWOTMBPlP5"

In [4]:
exp = Experiment (
    api_key="EpnUTFvwqbgvmSpjWOTMBPlP5",
    project_name="rmiranda-movie-reviews",
    workspace="rodrigomiranda21",
    # Some extra optional parameters below
    # auto_histogram_weight_logging=True,
    # log_code=True,
    )

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/rodrigomiranda21/rmiranda-movie-reviews/ae1cfb4ce42849e5a61586353a206682



In [5]:
 params = {
     'layer-1-size': 16,
     'epochs': 10,
     'batch-size':512,
     'dropout': 0.2,
     }

# This will be logged in the Hyperparameters Tab in the Comet UI
exp.log_parameters(params)

**Load Model**

The model will be used for fine tuning. It will be the first "layer" of our NN.

In [6]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(model, 
                           output_shape=[20], 
                           input_shape=[], 
                           dtype=tf.string, 
                           trainable=True)

In [ ]:
# hub_layer(train_examples[:3])

**Build Model**

Create Keras NN - Model + layer-1 + final layer (size 1) 

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(exp.get_parameter('layer-1-size'), activation='relu'))
model.add(tf.keras.layers.Dropout(exp.get_parameter('dropout')))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


**Train Model**

In [9]:
# After training the model you will see new Charts, Panels, Hyperparameters, Metrics and more in the Comet UI

model.fit(partial_x_train,
          partial_y_train,
          epochs=exp.get_parameter('epochs'),
          batch_size=exp.get_parameter('batch-size'),
          validation_data=(x_val, y_val),
          verbose=1)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/10
30/30 [==============================] - 3s 65ms/step - loss: 0.8677 - accuracy: 0.5368 - val_loss: 0.6683 - val_accuracy: 0.6024
Epoch 2/10
30/30 [==============================] - 2s 57ms/step - loss: 0.7092 - accuracy: 0.5813 - val_loss: 0.6329 - val_accuracy: 0.6493
Epoch 3/10
30/30 [==============================] - 2s 58ms/step - loss: 0.6423 - accuracy: 0.6298 - val_loss: 0.6071 - val_accuracy: 0.6794
Epoch 4/10
30/30 [==============================] - 2s 58ms/step - loss: 0.6127 - accuracy: 0.6687 - val_loss: 0.5848 - val_accuracy: 0.7078
Epoch 5/10
30/30 [==============================] - 2s 58ms/step - loss: 0.5845 - accuracy: 0.6945 - val_loss: 0.5612 - val_accuracy: 0.7321
Epoch 6/10
30/30 [==============================] - 2s 58ms/step - loss: 0.5570 - accuracy: 0.7189 - val_loss: 0.5358 - val_accuracy: 0.7505
Epoch 7/10
30/30 [==============================] - 2s 58ms/step - loss: 0.5309 - accuracy: 0.7374 - val_loss: 0.5091 - val_accuracy: 0.7705
Epoch 8/10
30

Logging some things about this experiment:

In [10]:
# log any custom Metric
exp.log_metric('my_custom_metric', 3.14)
    
# log a dataset hash (Hyperparameter)
exp.log_dataset_hash(partial_x_train)

In [11]:
# Logs text: 10 examples from the test dataset with it's labels. These strings appear on the Text Tab in the Comet UI.

for i in range(10):
    exp.log_text(train_examples[i], metadata={"label": train_labels[i].item()})

In [12]:
# Logs the Confusion Matrix

preds = model.predict(test_examples)

# Function to have the labels in vector format    
def onehot(val):
    retval = [0, 0]
    tmp = (val[0] + 1) / 2
    tmp = int(round(tmp))
    tmp = max(min(1, tmp), 0)
    retval[tmp] = 1
    return retval
    
new_preds = [onehot(v) for v in preds]

# Function to show text examples on the Confusion Matrix GUI    
def index_to_example(index):
    text = test_examples[index]
    # data = experiment.log_text(text)
    return {"sample": text.decode(), 
            "assetId": None,
            "type": "string"}
    
exp.log_confusion_matrix(new_preds, 
                         test_labels, 
                         index_to_example_function=index_to_example,
                         file_name="movie-reviews")

782/782 [==============================] - 3s 3ms/step


{'web': 'https://www.comet.com/api/asset/download?assetId=37295ba65b9b49df8697417b17d7f861&experimentKey=ae1cfb4ce42849e5a61586353a206682',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=37295ba65b9b49df8697417b17d7f861&experimentKey=ae1cfb4ce42849e5a61586353a206682',
 'assetId': '37295ba65b9b49df8697417b17d7f861'}

In [13]:
# Logs Model: The NN in h5 format

registry_model_name ='movie-reviews-nn'

model.save('models/movie-reviews-nn.h5')
exp.log_model(registry_model_name, 'models/movie-reviews-nn.h5')

{'web': 'https://www.comet.com/api/asset/download?assetId=307f7c3a7ae14ed293973cb6e73c100a&experimentKey=ae1cfb4ce42849e5a61586353a206682',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=307f7c3a7ae14ed293973cb6e73c100a&experimentKey=ae1cfb4ce42849e5a61586353a206682',
 'assetId': '307f7c3a7ae14ed293973cb6e73c100a'}

In [14]:
exp.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/rodrigomiranda21/rmiranda-movie-reviews/ae1cfb4ce42849e5a61586353a206682
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.5368000268936157, 0.8030666708946228)
COMET INFO:     batch_accuracy [30]          : (0.5305397510528564, 0.800409197807312)
COMET INFO:     batch_loss [30]              : (0.44278281927108765, 1.0209846496582031)
COMET INFO:     epoch_duration [10]          : (1.7004715739999767, 2.757266657999992)
COMET INFO:     loss [10]                    : (0.43832623958587646, 0.867737889289856)
COMET INFO:     my_custom_metric             : 3.14
COMET INFO:     val_accuracy [10]            : (0.602400004863739, 0.8148999810218811)
COMET INFO:     val_loss [10]                : (0.4303391575813

Register the model on Model Registry

In [15]:
api = API(api_key="EpnUTFvwqbgvmSpjWOTMBPlP5")

WORKSPACE = 'rodrigomiranda21'
PROJECT_NAME = 'rmiranda-movie-reviews'
EXP_KEY = exp.get_key()

api_exp = api.get_experiment(WORKSPACE, PROJECT_NAME, EXP_KEY)

api_exp.register_model(registry_model_name, version = "1.4.1")

COMET INFO: Successfully registered 'movie-reviews-nn', version '1.4.1' in workspace 'rodrigomiranda21'


{'registryModelId': 'OppQGGPpkHAvdP2d2237UafQ0',
 'registryModelItemId': 'O0oS5Q75Znl4kqYkdou96ZaS3'}